<table class="table table-bordered">
    <tr>
        <th style="text-align:center; vertical-align: middle; width:50%"><img src='https://www.np.edu.sg/images/default-source/default-album/img-logo.png'"></th>
        <th style="text-align:center;"><h1>Agile DataOps</h1><h2>Assignment 2(Group) - Data Cleansing</h2><h3>AY2024/25 Semester</h3></th>
    </tr>
</table>

In [2]:
##removing special characters
##table name : circuits

In [ ]:
SELECT 
    CIRCUITREF,
    NAME,
    -- allow hyphens by explicitly defining
    TRIM(REGEXP_REPLACE(LOCATION, '[^a-zA-Z\s-]', '')) AS FORMATTED_LOCATION
FROM 
    ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.circuits;

UPDATE ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.circuits
SET LOCATION = TRIM(REGEXP_REPLACE(LOCATION, '[^a-zA-Z\s-]', ''))
WHERE LOCATION != TRIM(REGEXP_REPLACE(LOCATION, '[^a-zA-Z\s-]', ''));

In [4]:
## table name : constructors

In [ ]:
SELECT 

    INITCAP(TRIM(REGEXP_REPLACE(
        REGEXP_REPLACE(CONSTRUCTORREF, '[-_]', ' '), 
        '([a-z])([A-Z])', 
        '\1 \2'
    ))) AS FORMATTED_CONSTRUCTORREF,

    
    INITCAP(TRIM(REGEXP_REPLACE(
        REGEXP_REPLACE(NAME, '[-_]', ' '), 
        '([a-z])([A-Z])', 
        '\1 \2'
    ))) AS FORMATTED_NAME

FROM 
    ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.CONSTRUCTOR;

UPDATE ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.CONSTRUCTOR
SET CONSTRUCTORREF = INITCAP(TRIM(REGEXP_REPLACE(
    REGEXP_REPLACE(CONSTRUCTORREF, '[-_]', ' '),
    '([a-z])([A-Z])',
    '\1 \2'
))),
NAME = INITCAP(TRIM(REGEXP_REPLACE(
    REGEXP_REPLACE(NAME, '[-_]', ' '),
    '([a-z])([A-Z])',
    '\1 \2'
)));


In [6]:
## since i only see both of the 2 shaded blanks

In [ ]:
UPDATE ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.CONSTRUCTOR
SET NAME = CASE 
    WHEN NAME LIKE '%█%' THEN 'McLaren Alfa Romeo'
    WHEN NAME LIKE '%Cooper █%' THEN 'Cooper Ferrari'
    ELSE NAME
END;

// table name : drivers

SELECT 
    INITCAP(TRIM(REGEXP_REPLACE(DRIVERREF, '[-_]', ' '))) AS FORMATTED_DRIVERREF
FROM 
    ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.DRIVERS;


UPDATE ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.DRIVERS
SET 
    DRIVERREF = INITCAP(TRIM(REGEXP_REPLACE(DRIVERREF, '[-_]', ''))),
    SURNAME = INITCAP(SURNAME);


In [8]:
## table name : lap times

In [ ]:
SELECT 
    TIME,
    -- Calculate total seconds
    CAST(SPLIT_PART(TIME, ':', 1) AS INT) * 60 + CAST(SPLIT_PART(TIME, ':', 2) AS FLOAT) AS TOTAL_SECONDS
FROM 
    ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.LAPTIMES;

ALTER TABLE ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.LAPTIMES ADD COLUMN TOTAL_SECONDS FLOAT;

UPDATE ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.LAPTIMES
SET TOTAL_SECONDS = CAST(SPLIT_PART(TIME, ':', 1) AS INT) * 60 + CAST(SPLIT_PART(TIME, ':', 2) AS FLOAT);


In [10]:
## table name : pitstops

In [ ]:
UPDATE ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.PITSTOPS
SET DURATION = (
    SELECT DURATION
    FROM ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.PITSTOPS
    WHERE DURATION IS NOT NULL
    ORDER BY RANDOM()
    LIMIT 1
)
WHERE DURATION IS NULL;


SELECT * 
FROM ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.PITSTOPS
WHERE DURATION IS NULL; -- no null results


In [12]:
## tablename : qualifying

In [ ]:
ALTER TABLE ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.QUALIFYING
ADD Q1_IN_SECONDS FLOAT, Q2_IN_SECONDS FLOAT, Q3_IN_SECONDS FLOAT;

-- Convert Q1, Q2, Q3 to seconds or replace invalid values with -1
UPDATE ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.QUALIFYING
SET 
    Q1_IN_SECONDS = CASE 
        WHEN Q1 IN ('DNQ', 'Unknown') THEN -1
        ELSE CAST(SPLIT_PART(Q1, ':', 1) AS INT) * 60 + CAST(SPLIT_PART(Q1, ':', 2) AS FLOAT)
    END,
    Q2_IN_SECONDS = CASE 
        WHEN Q2 IN ('DNQ', 'Unknown') THEN -1
        ELSE CAST(SPLIT_PART(Q2, ':', 1) AS INT) * 60 + CAST(SPLIT_PART(Q2, ':', 2) AS FLOAT)
    END,
    Q3_IN_SECONDS = CASE 
        WHEN Q3 IN ('DNQ', 'Unknown') THEN -1
        ELSE CAST(SPLIT_PART(Q3, ':', 1) AS INT) * 60 + CAST(SPLIT_PART(Q3, ':', 2) AS FLOAT)
    END;

SELECT 
    Q1, Q2, Q3, Q1_IN_SECONDS, Q2_IN_SECONDS, Q3_IN_SECONDS
FROM 
    ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.QUALIFYING;

UPDATE ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.QUALIFYING
SET 
    Q1_IN_SECONDS = COALESCE(Q1_IN_SECONDS, -1),
    Q2_IN_SECONDS = COALESCE(Q2_IN_SECONDS, -1),
    Q3_IN_SECONDS = COALESCE(Q3_IN_SECONDS, -1);

SELECT 
    Q1,
    Q2,
    Q3,
    CASE 
        WHEN Q1_IN_SECONDS = -1 THEN 'DNQ'
        ELSE CAST(Q1_IN_SECONDS AS VARCHAR)
    END AS Q1_IN_SECONDS,
    CASE 
        WHEN Q2_IN_SECONDS = -1 THEN 'DNQ'
        ELSE CAST(Q2_IN_SECONDS AS VARCHAR)
    END AS Q2_IN_SECONDS,
    CASE 
        WHEN Q3_IN_SECONDS = -1 THEN 'DNQ'
        ELSE CAST(Q3_IN_SECONDS AS VARCHAR)
    END AS Q3_IN_SECONDS
FROM 
    ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.QUALIFYING;


SELECT 
    Q1,
    Q2,
    Q3,
    -- Interpret -1 as 'DNQ' for Q1_IN_SECONDS
    CASE 
        WHEN Q1_IN_SECONDS = -1 THEN 'DNQ'
        ELSE CAST(Q1_IN_SECONDS AS VARCHAR)
    END AS Q1_IN_SECONDS_DISPLAY,
    -- Interpret -1 as 'DNQ' for Q2_IN_SECONDS
    CASE 
        WHEN Q2_IN_SECONDS = -1 THEN 'DNQ'
        ELSE CAST(Q2_IN_SECONDS AS VARCHAR)
    END AS Q2_IN_SECONDS_DISPLAY,
    -- Interpret -1 as 'DNQ' for Q3_IN_SECONDS
    CASE 
        WHEN Q3_IN_SECONDS = -1 THEN 'DNQ'
        ELSE CAST(Q3_IN_SECONDS AS VARCHAR)
    END AS Q3_IN_SECONDS_DISPLAY
FROM 
    ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.QUALIFYING;

//verify 
SELECT 
    COUNT(*) AS TOTAL_ROWS,
    COUNT(CASE WHEN Q1_IN_SECONDS = -1 THEN 1 END) AS Q1_DNQ_COUNT,
    COUNT(CASE WHEN Q2_IN_SECONDS = -1 THEN 1 END) AS Q2_DNQ_COUNT,
    COUNT(CASE WHEN Q3_IN_SECONDS = -1 THEN 1 END) AS Q3_DNQ_COUNT
FROM 
    ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.QUALIFYING;

In [14]:
## table name : result

In [ ]:
ALTER TABLE ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.RESULTS
ADD COLUMN FASTESTLAPTIME_SECONDS VARCHAR;


UPDATE ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.RESULTS
SET FASTESTLAPTIME_SECONDS = 
    CASE 
        WHEN FASTESTLAPTIME IS NULL OR FASTESTLAPTIME = '' OR FASTESTLAPTIME NOT LIKE '%:%' THEN 'NA'
        ELSE CAST(CAST(SPLIT_PART(FASTESTLAPTIME, ':', 1) AS INT) * 60 + CAST(SPLIT_PART(FASTESTLAPTIME, ':', 2) AS FLOAT) AS VARCHAR)
    END;

## verify  
SELECT FASTESTLAPTIME, FASTESTLAPTIME_SECONDS
FROM ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.RESULTS;


In [16]:
## table name : sprint_result_cleaned

In [ ]:
ALTER TABLE ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.SPRINT_RESULTS_CLEANED
ADD COLUMN TIME_IN_SECONDS VARCHAR;

ALTER TABLE ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.SPRINT_RESULTS_CLEANED
ADD COLUMN FASTESTLAPTIME_IN_SECONDS VARCHAR;

-- Replace NULL and empty strings in VARCHAR columns with 'NA'
UPDATE ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.SPRINT_RESULTS_CLEANED
SET 
    TIME = COALESCE(NULLIF(TRIM(TIME), ''), 'NA'),
    MILLISECONDS = COALESCE(NULLIF(TRIM(MILLISECONDS), ''), 'NA'),
    FASTESTLAP = COALESCE(NULLIF(TRIM(FASTESTLAP), ''), 'NA'),
    FASTESTLAPTIME = COALESCE(NULLIF(TRIM(FASTESTLAPTIME), ''), 'NA');


    
UPDATE ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.SPRINT_RESULTS_CLEANED
SET 
    TIME_IN_SECONDS = COALESCE(NULLIF(TRIM(TIME_IN_SECONDS), ''), 'NA'),
    FASTESTLAPTIME_IN_SECONDS = COALESCE(NULLIF(TRIM(FASTESTLAPTIME_IN_SECONDS), ''), 'NA');



-- Verify the updates

SELECT 
    TIME, 
    MILLISECONDS, 
    FASTESTLAP, 
    FASTESTLAPTIME, 
    TIME_IN_SECONDS, 
    FASTESTLAPTIME_IN_SECONDS
FROM ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.SPRINT_RESULTS_CLEANED
WHERE TIME_IN_SECONDS IS NULL OR FASTESTLAPTIME_IN_SECONDS IS NULL;



In [18]:
## status

In [ ]:
ALTER TABLE ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.STATUS
ADD COLUMN DESCRIPTIVE_STATUS VARCHAR(50);
UPDATE ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.STATUS
SET 
    DESCRIPTIVE_STATUS = CASE
        WHEN STATUS = '+1 Lap' THEN 'Additional 1 Lap'
        WHEN STATUS = '+2 Laps' THEN 'Additional 2 Laps'
        WHEN STATUS = '+3 Laps' THEN 'Additional 3 Laps'
        WHEN STATUS = '+4 Laps' THEN 'Additional 4 Laps'
        WHEN STATUS = '+5 Laps' THEN 'Additional 5 Laps'
        WHEN STATUS = '+6 Laps' THEN 'Additional 6 Laps'
        WHEN STATUS = '+7 Laps' THEN 'Additional 7 Laps'
        WHEN STATUS = '+8 Laps' THEN 'Additional 8 Laps'
        WHEN STATUS = '+9 Laps' THEN 'Additional 9 Laps'
        WHEN STATUS = '+10 Laps' THEN 'Additional 10 Laps'
        WHEN STATUS = '+11 Laps' THEN 'Additional 11 Laps'
        WHEN STATUS = '+12 Laps' THEN 'Additional 12 Laps'
        WHEN STATUS = '+13 Laps' THEN 'Additional 13 Laps'
        WHEN STATUS = '+14 Laps' THEN 'Additional 14 Laps'
        WHEN STATUS = '+15 Laps' THEN 'Additional 15 Laps'
        WHEN STATUS = '+16 Laps' THEN 'Additional 16 Laps'
        WHEN STATUS = '+17 Laps' THEN 'Additional 17 Laps'
        WHEN STATUS = '+18 Laps' THEN 'Additional 18 Laps'
        WHEN STATUS = '+19 Laps' THEN 'Additional 19 Laps'
        WHEN STATUS = '+20 Laps' THEN 'Additional 20 Laps'
        WHEN STATUS = '+21 Laps' THEN 'Additional 21 Laps'
        WHEN STATUS = '+22 Laps' THEN 'Additional 22 Laps'
        WHEN STATUS = '+23 Laps' THEN 'Additional 23 Laps'
        WHEN STATUS = '+24 Laps' THEN 'Additional 24 Laps'
        WHEN STATUS = '+25 Laps' THEN 'Additional 25 Laps'
        WHEN STATUS = '+26 Laps' THEN 'Additional 26 Laps'
        WHEN STATUS = '+27 Laps' THEN 'Additional 27 Laps'
        WHEN STATUS = '+28 Laps' THEN 'Additional 28 Laps'
        WHEN STATUS = '+29 Laps' THEN 'Additional 29 Laps'
        WHEN STATUS = '+30 Laps' THEN 'Additional 30 Laps'
        WHEN STATUS = '+31 Laps' THEN 'Additional 31 Laps'
        WHEN STATUS = '+32 Laps' THEN 'Additional 32 Laps'
        WHEN STATUS = '+33 Laps' THEN 'Additional 33 Laps'
        WHEN STATUS = '+34 Laps' THEN 'Additional 34 Laps'
        WHEN STATUS = '+35 Laps' THEN 'Additional 35 Laps'
        WHEN STATUS = '+36 Laps' THEN 'Additional 36 Laps'
        WHEN STATUS = '+37 Laps' THEN 'Additional 37 Laps'
        WHEN STATUS = '+38 Laps' THEN 'Additional 38 Laps'
        WHEN STATUS = '+39 Laps' THEN 'Additional 39 Laps'
        WHEN STATUS = '+40 Laps' THEN 'Additional 40 Laps'
        WHEN STATUS = '+41 Laps' THEN 'Additional 41 Laps'
        WHEN STATUS = '+42 Laps' THEN 'Additional 42 Laps'
        WHEN STATUS = '+43 Laps' THEN 'Additional 43 Laps'
        WHEN STATUS = '+44 Laps' THEN 'Additional 44 Laps'
        WHEN STATUS = '+45 Laps' THEN 'Additional 45 Laps'
        WHEN STATUS = '+46 Laps' THEN 'Additional 46 Laps'
        WHEN STATUS = '+47 Laps' THEN 'Additional 47 Laps'
        WHEN STATUS = '+48 Laps' THEN 'Additional 48 Laps'
        WHEN STATUS = '+49 Laps' THEN 'Additional 49 Laps'
        ELSE STATUS
    END;
//VERIFY
SELECT * FROM ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.STATUS;


#### Additional Addings

In [ ]:
-- Add the positionText column to DriverStandings table
ALTER TABLE ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.DRIVER_STANDINGS
ADD COLUMN positionText VARCHAR(20);

-- Add the positionText column to LapTimes table
ALTER TABLE ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.LAPTIMES
ADD COLUMN positionText VARCHAR(20);

-- Add the positionText column to Qualifying table
ALTER TABLE ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.QUALIFYING
ADD COLUMN positionText VARCHAR(20);

In [23]:
## Update the positionText in DriverStandings

In [ ]:
UPDATE ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.DRIVER_STANDINGS
SET positionText = CASE 
    WHEN OVERALLPOSITION = 1 THEN 'Winner'
    WHEN OVERALLPOSITION = 2 THEN 'Podium 2'
    WHEN OVERALLPOSITION = 3 THEN 'Podium 3'
    ELSE CONCAT('P', OVERALLPOSITION)
END;


In [ ]:
## Update the positionText in LapTimes

In [ ]:
UPDATE ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.LAPTIMES
SET positionText = CASE 
    WHEN position = 1 THEN 'Fastest Lap'
    ELSE CONCAT('P', position)
END;


In [ ]:
## Update the positionText in Qualifying

In [ ]:
UPDATE ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.QUALIFYING
SET positionText = CASE 
    WHEN position = 1 THEN 'Pole'
    ELSE CONCAT('P', position)
END;

In [ ]:
## Update the descriptive_status

In [ ]:
UPDATE ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.STATUS
SET 
    DESCRIPTIVE_STATUS = CASE
        WHEN STATUS = 'ERS' THEN 'Energy Recovery'
        WHEN STATUS = 'DNF' THEN 'Did Not Finish'
        WHEN STATUS = 'DNS' THEN 'Did Not Start'
        WHEN STATUS = 'PIT' THEN 'Pit Stop'
        WHEN STATUS = 'CV Joint' THEN 'Constant Velocity Joint'
        ELSE STATUS
    END;

In [25]:
## To verify

In [ ]:
## Verify updates in DriverStandings
SELECT DRIVERSTANDINGSID, positionText FROM ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.DRIVER_STANDINGS;

##Verify updates in LapTimes
SELECT raceId, driverId, positionText FROM ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.LAPTIMES;

## Verify updates in Qualifying
SELECT qualifyId, raceId, driverId, positionText FROM ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.QUALIFYING;

## Verify updates in Status
SELECT STATUSID, status FROM ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.STATUS;


In [ ]:
## handling data types (minor change)

-- Update DOB to DATE
ALTER TABLE ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.SEASONS ADD YEAR_NEW NUMBER(4,0);
UPDATE ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.SEASONS
SET YEAR_NEW = CAST(YEAR AS NUMBER(4,0));
ALTER TABLE ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.SEASONS DROP COLUMN YEAR;
ALTER TABLE ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.SEASONS RENAME COLUMN YEAR_NEW TO YEAR;

SELECT * FROM ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.SEASONS;

## dropping
ALTER TABLE ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.DRIVERS
DROP COLUMN DOB_DATE;

In [ ]:
## sprint_results encoding

In [ ]:
UPDATE ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.SPRINT_RESULTS
SET FASTESTLAPTIME_IN_SECONDS = CASE 
    WHEN FASTESTLAPTIME IN ('NA', 'DNQ', 'NULL', '') THEN NULL
    WHEN FASTESTLAPTIME LIKE '%:%' THEN 
        CAST(SPLIT_PART(FASTESTLAPTIME, ':', 1) AS INT) * 60 + CAST(SPLIT_PART(SPLIT_PART(FASTESTLAPTIME, ':', 2), '.', 1) AS FLOAT)
    WHEN FASTESTLAPTIME LIKE '%.%' THEN CAST(FASTESTLAPTIME AS FLOAT)
    ELSE NULL
END;


UPDATE ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.SPRINT_RESULTS
SET 
    TIME_IN_SECONDS = CASE 
        WHEN TIME_IN_SECONDS IS NULL THEN -1
        ELSE TIME_IN_SECONDS
    END;

UPDATE ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.SPRINT_RESULTS
SET 
    FASTESTLAPTIME_IN_SECONDS = CASE 
        WHEN FASTESTLAPTIME_IN_SECONDS IS NULL THEN -1
        ELSE FASTESTLAPTIME_IN_SECONDS
    END;


SELECT 
    TIME, 
    MILLISECONDS, 
    FASTESTLAPTIME, 
    TIME_IN_SECONDS, 
    FASTESTLAPTIME_IN_SECONDS
FROM ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.SPRINT_RESULTS
WHERE TIME_IN_SECONDS IS NULL OR FASTESTLAPTIME_IN_SECONDS IS NULL;


In [29]:
## result_cleaned( replace results, changed NA to DNS, did not set)

In [ ]:
CREATE OR REPLACE TABLE ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.RESULT_CLEANED (
    RESULTID NUMBER(38,0) PRIMARY KEY,
    RACEID NUMBER(38,0),
    DRIVERID NUMBER(38,0),
    CONSTRUCTORID NUMBER(38,0),
    DRIVERNUM NUMBER(38,1),
    POSITIONTEXT VARCHAR(16777216),
    POSITIONORDER NUMBER(38,0),
    POINTS NUMBER(38,0),
    LAPS NUMBER(38,0),
    TIME VARCHAR(16777216),
    MILLISECONDS VARCHAR(16777216),
    FASTESTLAP VARCHAR(16777216),
    RANK VARCHAR(16777216),
    FASTESTLAPTIME VARCHAR(16777216),
    FASTESTLAPSPEED VARCHAR(16777216),
    STATUSID NUMBER(38,0),
    RN NUMBER(18,0),
    FASTESTLAPTIME_SECONDS VARCHAR(16777216),
    CONSTRAINT FK_RESULT_CLEANED_RACES FOREIGN KEY (RACEID) REFERENCES ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.RACES(RACEID),
    CONSTRAINT FK_RESULT_CLEANED_DRIVERS FOREIGN KEY (DRIVERID) REFERENCES ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.DRIVERS(DRIVERID),
    CONSTRAINT FK_RESULT_CLEANED_CONSTRUCTORS FOREIGN KEY (CONSTRUCTORID) REFERENCES ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.CONSTRUCTOR(CONSTRUCTORID),
    CONSTRAINT FK_RESULT_CLEANED_STATUS FOREIGN KEY (STATUSID) REFERENCES ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.STATUS(STATUSID)
);

INSERT INTO ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.RESULT_CLEANED
SELECT *
FROM ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.RESULTS;

UPDATE ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.RESULT_CLEANED
SET POSITIONTEXT = 'DNS'
WHERE POSITIONTEXT = 'NA';

UPDATE ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.RESULT_CLEANED
SET FASTESTLAP = 'DNS'
WHERE FASTESTLAP = 'NA';

UPDATE ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.RESULT_CLEANED
SET FASTESTLAPTIME = 'DNS'
WHERE FASTESTLAPTIME = 'NA';

UPDATE ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.RESULT_CLEANED
SET FASTESTLAPSPEED = 'DNS'
WHERE FASTESTLAPSPEED = 'NA';

UPDATE ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.RESULT_CLEANED
SET FASTESTLAPTIME_SECONDS = 'DNS'
WHERE FASTESTLAPTIME_SECONDS = 'NA';

UPDATE ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.RESULT_CLEANED
SET TIME = 'DNS'
WHERE TIME = 'NA';

UPDATE ADO_ASG2_GROUP_CAT_DB.ADO_ASG2_GROUP_CAT_SCM.RESULT_CLEANED
SET MILLISECONDS = 'DNS'
WHERE MILLISECONDS = 'NA';